Initiation of libraries and the mtcnn face detection pipeline

In [ ]:
import os
import cv2
import glob, random
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
import sklearn

print("Setting up detection pipeline.\n")

# If required, create a face detection pipeline using MTCNN:
mtcnn = MTCNN(image_size=160, margin=0)

# Create an inception resnet (in eval mode):
resnet = InceptionResnetV1(pretrained='vggface2').eval()

Create Embeddings for Database

In [ ]:
path = "./data/test_images/"
database = os.listdir(path)

print("\n\nFollowing persons can be recognised from the given dataset:")
print(database)

############################################################################
# name_dict = create_dict_with_embeddings(path, database, resnet)
##############
print("\nEmbeddings are being calculated", end="")

name_dict = {}

for person in database:
    tensor_list = []
    images = os.listdir(path + person)
    for image in images:
        # Open Image in person-folder
        db_img = Image.open(path + person + '/' + image)

        # Get cropped and prewhitened image tensor
        db_img_cropped = mtcnn(db_img)

        # Calculate embedding (unsqueeze to add batch dimension)
        db_img_embedding = resnet(db_img_cropped.unsqueeze(0))

        # Save embeddings in 'tensor_list'
        tensor_list.append(db_img_embedding)
    # Save tensor_list for a person in 'name_dict'
    name_dict[person] = tensor_list

    print(".", end="")

print("\n\nFinished!")
############################################################################

Create standard embedding from input data and Face Detection

In [ ]:
input_path_type = ["./data/input_data/*.jpg"]
input_images = glob.glob(random.choice(input_path_type))
random_image = random.choice(input_images)
input_img = Image.open(random_image)

crop_path = "./pytorch_save/img_1.jpg"

# Get cropped and prewhitened image tensor
input_img_cropped = mtcnn(input_img, save_path=crop_path)

# Calculate embedding (unsqueeze to add batch dimension)
input_img_embedding = resnet(input_img_cropped.unsqueeze(0))

boxes, probs, landmarks = mtcnn.detect(input_img, landmarks=True)

fig, axarray = plt.subplots(1, 2, figsize=(8, 6))
axarray[0].imshow(input_img)
axarray[0].axis('off')
axarray[1].imshow(input_img)
axarray[1].axis('off')

for box, landmark in zip(boxes, landmarks):
    axarray[1].scatter(*np.meshgrid(box[[0, 2]], box[[1, 3]]))
    axarray[1].scatter(landmark[:, 0], landmark[:, 1], s=8)
fig.show()

Cosine Similarity --> Face Recognition

In [ ]:
cos = torch.nn.CosineSimilarity()
best = 0
for person in name_dict:
    print(person, ": ", end='')
    img_index = 1
    for tensor in name_dict[person]:
        out = cos(tensor, input_img_embedding)
        val = out.detach().numpy()[0]
        print(val)
        if val > best:
            best = val
            best_list = [person, best, tensor, img_index]
            img_index = +1
print("\nBest Similarity: ", best_list[0], ",", best_list[1], ",", "#Image:", best_list[3])

threshold = 0.6
print("\nValue must remain smaller than threshold = ", threshold)
if best_list[1] > threshold:
    print("\nPerson is " + best_list[0])

    fig, axarr = plt.subplots(1, 2, figsize=(8, 6))
    axarr[0].imshow(input_img)
    axarr[0].axis('off')
    axarr[1].imshow(Image.open('./data/test_images/' + best_list[0] + "/" + str(best_list[3]) + ".jpg"))
    axarr[1].axis('off')
    fig.show()
else:
    print("\nPerson is unknown.")

From here starts the Explainability Pipeline

Rastering the cropped image

In [ ]:
from clusteringv15 import cluster_face

img = Image.open(crop_path)

# clear savepath
print("Clear savepath.")
chop_path = './chopped_img_temp/'
chop_data = os.listdir(chop_path)
if chop_data[0] == '.ipynb_checkpoints' and not os.listdir(chop_path):
    chop_data.remove('.ipynb_checkpoints')
for f in chop_data:
    os.remove(os.path.join(chop_path, f))

# generate faceclusters and return chops
chops = cluster_face(img, denominator=8, savepath=chop_path)

chop_data = os.listdir(chop_path)
if chop_data[0] == '.ipynb_checkpoints':
    chop_data.remove('.ipynb_checkpoints')

Add noise to Image pieces

In [ ]:
from clusteringv15 import add_noise

from cluster_embeddings import cluster_embeddings

In [ ]:
noise_factors = np.arange(0, 1, 0.1)
comp_dict = {}

new_comps = cluster_embeddings(img, noise_factors, chop_data, chop_path, chops, mtcnn, resnet)
comp_dict.update(new_comps)
print(comp_dict.keys())

In [ ]:
# Plot random Images
no = 8
fig, axarray = plt.subplots(1, no, figsize=(15, 15))
noise_path_type = ['./noise_chops/*.jpg']
m = 0
while m != no:
    noise_image = glob.glob(random.choice(noise_path_type))
    random_image = random.choice(noise_image)
    rndm = Image.open(random_image)
    axarray[m].imshow(rndm)
    axarray[m].axis('off')
    m += 1
fig.show()
plt.savefig("./noise_images.jpg")

PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn import preprocessing
############################################################################
# score = plot_pca(comp_dict, plt)
#################
score = []
pca = PCA()
t = 0
for comp in comp_dict:
    X = comp_dict[comp]

    pca.fit(X)
    pca_data = pca.transform(X)

    per_var = np.round(pca.explained_variance_ratio_ * 100, decimals=1)
    labels = ['PC' + str(x) for x in range(1, len(per_var) + 1)]

    plt.bar(x=range(1, len(per_var) + 1), height=per_var, tick_label=labels)
    plt.ylabel('Percentage of Explained Variance')
    plt.xlabel('Principal Component')
    plt.title(comp)
    plt.show()

    print(comp + ': ' + str(pca.explained_variance_ratio_[0] * 100))
    score.append(pca.explained_variance_ratio_[0])
############################################################################

Heatmap

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib import cm as cmx

############################################################################
# plot_heat_map(chops, score)
###############################
pic = np.zeros((160, 160))
i = 0
for clust in chops:
    x = clust[0]
    while (x - 1) < clust[2]:
        y = clust[1]
        while (y - 1) < clust[3]:
            pic[x, y] = 1 - score[i]
            y += 1
        x += 1
    i += 1
cnorm = colors.Normalize(vmin=0, vmax=1)
org_img = plt.imshow(Image.open('./pytorch_save/img_1.jpg'))
heatmap = plt.imshow(pic, cmap='inferno', norm=cnorm, interpolation='nearest', alpha=0.6)
plt.show()
############################################################################